In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

RAW_DATA_PATH = r"../data/raw"
PROCESSED_DATA_PATH = r"../data/processed"

In [3]:
plt.style.use("seaborn-v0_8-whitegrid")
sns.set_palette("Set2")
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 12

pd.set_option("display.max_columns", None)

In [4]:
covid_data = pd.read_csv(os.path.join(RAW_DATA_PATH, "covid_19_data.csv"))

In [5]:
global_time_series_files = [
    'time_series_covid_19_confirmed.csv',
    'time_series_covid_19_deaths.csv',
    'time_series_covid_19_recovered.csv'
]

In [6]:
global_time_series_data = {}
for file in global_time_series_files:
    df = pd.read_csv(os.path.join(RAW_DATA_PATH, file))
    key = file.replace("time_series_covid_19_", "").replace(".csv", "")
    global_time_series_data[key] = df

    print(f"{key}: {df.shape}")

confirmed: (276, 498)
deaths: (276, 498)
recovered: (261, 498)


In [7]:
print("\n--- THÔNG TIN VỀ DỮ LIỆU TỔNG HỢP COVID-19 ---")
    
# Hiển thị thông tin cơ bản
print("Columns:", covid_data.columns.tolist())
print("\nThông tin cơ bản về dataset:")
print(covid_data.info())

# Kiểm tra missing values
missing_values = covid_data.isnull().sum()
if missing_values.sum() > 0:
    print("\nGiá trị bị thiếu trong dữ liệu:")
    print(missing_values[missing_values > 0])
else:
    print("\nKhông có giá trị bị thiếu trong dữ liệu.")

# Kiểm tra phạm vi thời gian
if 'ObservationDate' in covid_data.columns:
    covid_data['ObservationDate'] = pd.to_datetime(covid_data['ObservationDate'])
    min_date = covid_data['ObservationDate'].min()
    max_date = covid_data['ObservationDate'].max()
    print(f"\nPhạm vi thời gian: {min_date.strftime('%Y-%m-%d')} đến {max_date.strftime('%Y-%m-%d')}")
    print(f"Tổng số ngày: {(max_date - min_date).days + 1}")

# Kiểm tra giá trị âm trong dữ liệu tổng hợp
print("\n--- KIỂM TRA CHẤT LƯỢNG DỮ LIỆU ---")
cols_to_check = ['Confirmed', 'Deaths', 'Recovered']

for col in cols_to_check:
    if col in covid_data.columns:
        neg_values = covid_data[covid_data[col] < 0]
        if len(neg_values) > 0:
            print(f"\nTìm thấy {len(neg_values)} giá trị âm trong cột {col}:")
            print(neg_values[['ObservationDate', 'Country/Region', 'Province/State', col]].head(10))
            
            # Phân tích thêm về giá trị âm
            print(f"\nPhân tích giá trị âm trong cột {col}:")
            neg_by_country = neg_values.groupby('Country/Region')[col].count().sort_values(ascending=False)
            print(f"Top 5 quốc gia có nhiều giá trị âm nhất:")
            print(neg_by_country.head(5))
            
            # Tính tỷ lệ giá trị âm
            neg_ratio = len(neg_values) / len(covid_data) * 100
            print(f"Tỷ lệ giá trị âm: {neg_ratio:.2f}% của tổng số dữ liệu")
        else:
            print(f"Không tìm thấy giá trị âm trong cột {col}")

# Kiểm tra giá trị 'Unknown' trong Province/State
if 'Province/State' in covid_data.columns:
    unknown_province = covid_data[covid_data['Province/State'].str.contains('Unknown', na=False, case=False)]
    
    if len(unknown_province) > 0:
        print(f"\nTìm thấy {len(unknown_province)} hàng có giá trị 'Unknown' trong cột Province/State:")
        print(unknown_province[['ObservationDate', 'Country/Region', 'Province/State']].head(10))
        
        # Phân tích tác động
        unknown_by_country = unknown_province.groupby('Country/Region').size().sort_values(ascending=False)
        print("\nSố lượng giá trị 'Unknown' theo quốc gia:")
        print(unknown_by_country.head(10))
        
        # Tính tỷ lệ Unknown
        unknown_ratio = len(unknown_province) / len(covid_data) * 100
        print(f"Tỷ lệ giá trị 'Unknown': {unknown_ratio:.2f}% của tổng số dữ liệu")
    else:
        print("\nKhông tìm thấy giá trị 'Unknown' trong cột Province/State")

# Các quốc gia và khu vực trong dữ liệu
if 'Country/Region' in covid_data.columns:
    country_column = 'Country/Region'
elif 'Country_Region' in covid_data.columns:
    country_column = 'Country_Region'
else:
    country_column = None

if country_column:
    num_countries = covid_data[country_column].nunique()
    print(f"\nSố lượng quốc gia/vùng lãnh thổ: {num_countries}")
    print("\nTop 10 quốc gia với số ca nhiễm nhiều nhất (theo dữ liệu gần nhất):")
    
    # Lấy dữ liệu ngày cuối cùng
    latest_date = covid_data['ObservationDate'].max()
    latest_data = covid_data[covid_data['ObservationDate'] == latest_date]
    
    # Tổng hợp theo quốc gia
    if 'Confirmed' in covid_data.columns:
        country_totals = latest_data.groupby(country_column)['Confirmed'].sum().sort_values(ascending=False)
        print(country_totals.head(10))

# Hiển thị một số thống kê mô tả
if 'Confirmed' in covid_data.columns and 'Deaths' in covid_data.columns and 'Recovered' in covid_data.columns:
    print("\nThống kê mô tả cho các số liệu chính:")
    print(covid_data[['Confirmed', 'Deaths', 'Recovered']].describe())


--- THÔNG TIN VỀ DỮ LIỆU TỔNG HỢP COVID-19 ---
Columns: ['SNo', 'ObservationDate', 'Province/State', 'Country/Region', 'Last Update', 'Confirmed', 'Deaths', 'Recovered']

Thông tin cơ bản về dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306429 entries, 0 to 306428
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   SNo              306429 non-null  int64  
 1   ObservationDate  306429 non-null  object 
 2   Province/State   228326 non-null  object 
 3   Country/Region   306429 non-null  object 
 4   Last Update      306429 non-null  object 
 5   Confirmed        306429 non-null  float64
 6   Deaths           306429 non-null  float64
 7   Recovered        306429 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 18.7+ MB
None

Giá trị bị thiếu trong dữ liệu:
Province/State    78103
dtype: int64

Phạm vi thời gian: 2020-01-22 đến 2021-05-29
Tổng số ngày: 494

--- KIỂM TRA CHẤ

In [8]:
print("\n--- THÔNG TIN VỀ DỮ LIỆU TIME SERIES TOÀN CẦU ---")

data_quality_issues = []  # Lưu trữ các vấn đề chất lượng dữ liệu

for key, df in global_time_series_data.items():
    print(f"\n{key.upper()}:")
    print("Columns (5 đầu tiên và 5 cuối cùng):", df.columns[:5].tolist(), "...", df.columns[-5:].tolist())
    
    # Kiểm tra cấu trúc
    print(f"Kích thước dataset: {df.shape}")
    
    # Số lượng quốc gia/vùng lãnh thổ
    num_countries = df['Country/Region'].nunique()
    num_provinces = df['Province/State'].notna().sum()
    
    print(f"Số lượng quốc gia/vùng lãnh thổ: {num_countries}")
    print(f"Số lượng tỉnh/tiểu bang có dữ liệu: {num_provinces}")
    
    # Phạm vi thời gian
    date_columns = [col for col in df.columns if '/' in col or '-' in col]
    start_date = date_columns[0]
    end_date = date_columns[-1]
    print(f"Phạm vi thời gian: {start_date} đến {end_date} ({len(date_columns)} ngày)")
    
    # Top 5 quốc gia có số ca nhiễm/tử vong/hồi phục cao nhất (theo cột cuối cùng)
    last_date = df.columns[-1]
    top_countries = df.groupby('Country/Region')[last_date].sum().sort_values(ascending=False).head(5)
    print(f"Top 5 quốc gia có số {key} cao nhất (tính đến {last_date}):")
    print(top_countries)
    
    # Kiểm tra missing values
    missing_values = df.isnull().sum()
    if missing_values.sum() > 0:
        print("\nGiá trị bị thiếu trong dữ liệu:")
        print(missing_values[missing_values > 0])
    else:
        print("\nKhông có giá trị bị thiếu trong các cột chính.")
    
    # KIỂM TRA CHẤT LƯỢNG DỮ LIỆU
    print(f"\n--- KIỂM TRA CHẤT LƯỢNG DỮ LIỆU {key.upper()} ---")
    
    # 1. Kiểm tra giá trị âm
    # Chuyển đổi các cột ngày thành dạng số trước khi so sánh
    numeric_df = df[date_columns].apply(pd.to_numeric, errors='coerce')

    # Kiểm tra giá trị âm
    neg_mask = (numeric_df < 0).any(axis=1)
    neg_rows = df[neg_mask]

    if len(neg_rows) > 0:
        print(f"\n1. Tìm thấy {len(neg_rows)} hàng có giá trị âm:")
        
        # Hiển thị thông tin các hàng có giá trị âm
        for idx, row in neg_rows.head(3).iterrows():  # Chỉ hiển thị 3 hàng đầu tiên để tiết kiệm không gian
            country = row['Country/Region']
            province = row['Province/State'] if not pd.isna(row['Province/State']) else 'N/A'
            
            # Tìm các cột có giá trị âm (phải dùng numeric_df để so sánh)
            neg_dates = [date for date in date_columns if pd.to_numeric(row[date], errors='coerce') < 0]
            
            print(f"  - {country}, {province}:")
            for date in neg_dates[:3]:  # Hiển thị tối đa 3 ngày có giá trị âm
                print(f"    {date}: {row[date]}")
            
            if len(neg_dates) > 3:
                print(f"    ... và {len(neg_dates) - 3} ngày khác")
        
        # Tính tổng số giá trị âm và tác động
        total_neg_values = (numeric_df < 0).sum().sum()
        total_values = len(df) * len(date_columns)
        neg_ratio = total_neg_values / total_values * 100
        
        print(f"\nTổng số giá trị âm: {total_neg_values}")
        print(f"Tỷ lệ: {neg_ratio:.4f}% của tổng số giá trị")
        
        # Lưu thông tin vấn đề chất lượng
        data_quality_issues.append({
            'File': f'time_series_covid_19_{key}.csv',
            'Issue': 'Negative values',
            'Count': total_neg_values,
            'Percentage': f"{neg_ratio:.4f}%",
            'Impact': 'May affect trend analysis and daily new cases calculation'
        })
        
        # Vẽ biểu đồ để minh họa các giá trị âm (chọn quốc gia đầu tiên có giá trị âm)
        if len(neg_rows) > 0:
            example_country = neg_rows.iloc[0]['Country/Region']
            example_province = neg_rows.iloc[0]['Province/State']
            
            # Lọc dữ liệu cho quốc gia/tỉnh này
            if pd.isna(example_province):
                example_data = numeric_df[df['Country/Region'] == example_country].iloc[0]
            else:
                example_data = numeric_df[(df['Country/Region'] == example_country) & 
                                (df['Province/State'] == example_province)].iloc[0]
            
            # Chuyển đổi thành DataFrame với index là ngày
            example_df = pd.DataFrame({'Value': example_data})
            example_df.index = pd.to_datetime(example_df.index, errors='coerce')
            
            # Loại bỏ các hàng có index là NaT (Not a Time)
            example_df = example_df.dropna(subset=['Value'])
            example_df = example_df[~example_df.index.isna()]
            
            # Sắp xếp theo index để đảm bảo tính toán diff chính xác
            example_df = example_df.sort_index()
            
            # Tính số ca mới mỗi ngày
            example_df['New'] = example_df['Value'].diff()
            
            # Vẽ biểu đồ
            plt.figure(figsize=(14, 6))
            
            plt.subplot(1, 2, 1)
            plt.plot(example_df.index, example_df['Value'], 'b-')
            plt.title(f'Tích lũy {key} - {example_country} {example_province if not pd.isna(example_province) else ""}')
            plt.xticks(rotation=45)
            plt.grid(True)
            
            plt.subplot(1, 2, 2)
            # Đánh dấu các giá trị âm bằng màu đỏ
            mask_pos = example_df['New'] >= 0
            mask_neg = example_df['New'] < 0
            
            plt.bar(example_df.index[mask_pos], example_df['New'][mask_pos], color='blue', label='Positive')
            plt.bar(example_df.index[mask_neg], example_df['New'][mask_neg], color='red', label='Negative')
            plt.title(f'Số {key} mới mỗi ngày (có thể âm)')
            plt.xticks(rotation=45)
            plt.grid(True)
            plt.legend()
            
            plt.tight_layout()
            plt.savefig(f'../Datasets/Processed/negative_values_{key}.png', dpi=300)
            plt.show()
    else:
        print(f"1. Không tìm thấy giá trị âm trong dữ liệu {key}")
    
    # 2. Kiểm tra 'Unknown' trong Province/State
    if 'Province/State' in df.columns:
        unknown_rows = df[df['Province/State'].str.contains('Unknown', na=False, case=False)]
        
        if len(unknown_rows) > 0:
            print(f"\n2. Tìm thấy {len(unknown_rows)} hàng có giá trị 'Unknown' trong Province/State:")
            
            # Hiển thị thông tin
            print(unknown_rows[['Country/Region', 'Province/State']].head(10))
            
            # Kiểm tra tác động đến tổng số ca
            if len(date_columns) > 0:
                last_date = date_columns[-1]
                total_unknown = unknown_rows[last_date].sum()
                total_all = df[last_date].sum()
                percentage = (total_unknown / total_all * 100) if total_all > 0 else 0
                
                print(f"\nTổng số {key} từ các hàng 'Unknown' (tính đến {last_date}): {total_unknown:,.0f}")
                print(f"Chiếm {percentage:.2f}% tổng số {key} toàn cầu ({total_all:,.0f})")
                
                # Lưu thông tin vấn đề chất lượng
                data_quality_issues.append({
                    'File': f'time_series_covid_19_{key}.csv',
                    'Issue': 'Unknown Province/State',
                    'Count': len(unknown_rows),
                    'Percentage': f"{percentage:.2f}%",
                    'Impact': 'May affect provincial/state level analysis'
                })
        else:
            print(f"2. Không tìm thấy giá trị 'Unknown' trong Province/State")

# Tạo DataFrame tổng hợp các vấn đề chất lượng dữ liệu và lưu
if data_quality_issues:
    quality_issues_df = pd.DataFrame(data_quality_issues)
    quality_issues_df.to_csv(os.path.join(PROCESSED_DATA_PATH, 'data_quality_issues.csv'), index=False)
    print(f"\nĐã lưu tổng hợp các vấn đề chất lượng dữ liệu vào: {PROCESSED_DATA_PATH}/data_quality_issues.csv")
    print("\nTổng hợp các vấn đề chất lượng dữ liệu:")
    print(quality_issues_df)
    
    # Hiển thị đề xuất xử lý
    print("""
    Đề xuất xử lý vấn đề chất lượng dữ liệu:
    
    1. Giá trị âm trong dữ liệu:
       - Nguyên nhân: Thường do hiệu chỉnh số liệu, phân loại lại các ca bệnh, hoặc lỗi báo cáo.
       - Phương pháp xử lý:
         * Khi tính số ca mới: Sử dụng max(0, diff) để tránh giá trị âm
         * Hoặc sử dụng trung bình trượt 7 ngày để làm mượt dữ liệu
         * Đánh dấu rõ các giá trị bất thường trong trực quan hóa
    
    2. Giá trị "Unknown" trong Province/State:
       - Nguyên nhân: Không xác định được vị trí cụ thể trong quốc gia
       - Phương pháp xử lý:
         * Đối với phân tích cấp quốc gia: Vẫn bao gồm các giá trị "Unknown"
         * Đối với phân tích cấp tỉnh/bang: Cần ghi chú rõ về tỷ lệ dữ liệu chưa xác định
    
    Các vấn đề này sẽ được xử lý chi tiết trong phần làm sạch dữ liệu (Ngày 3-4).
    """)


--- THÔNG TIN VỀ DỮ LIỆU TIME SERIES TOÀN CẦU ---

CONFIRMED:
Columns (5 đầu tiên và 5 cuối cùng): ['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20'] ... ['5/25/21', '5/26/21', '5/27/21', '5/28/21', '5/29/21']
Kích thước dataset: (276, 498)
Số lượng quốc gia/vùng lãnh thổ: 193
Số lượng tỉnh/tiểu bang có dữ liệu: 86
Phạm vi thời gian: Province/State đến 5/29/21 (496 ngày)
Top 5 quốc gia có số confirmed cao nhất (tính đến 5/29/21):
Country/Region
US        33251939
India     27894800
Brazil    16471600
France     5719877
Turkey     5235978
Name: 5/29/21, dtype: int64

Giá trị bị thiếu trong dữ liệu:
Province/State    190
Lat                 2
Long                2
dtype: int64

--- KIỂM TRA CHẤT LƯỢNG DỮ LIỆU CONFIRMED ---
1. Không tìm thấy giá trị âm trong dữ liệu confirmed

2. Tìm thấy 1 hàng có giá trị 'Unknown' trong Province/State:
   Country/Region Province/State
88          China        Unknown

Tổng số confirmed từ các hàng 'Unknown' (tính đến 5/29/21): 0
Chiếm 0.00%

In [9]:
confirmed_df = global_time_series_data.get("confirmed")
deaths_df = global_time_series_data.get("deaths")
recovered_df = global_time_series_data.get("recovered")